In [1]:
import os.path
import tensorflow as tf
import helper
import warnings
from distutils.version import LooseVersion
import project_tests as tests


In [2]:
# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))




TensorFlow Version: 1.2.1


C:\Users\gautam_sareen\AppData\Local\Continuum\Miniconda3\envs\carnd-term1\lib\site-packages\ipykernel\__main__.py:7: UserWarning: No GPU found. Please use a GPU to train your neural network.


In [3]:
def load_vgg(sess, vgg_path):
    """
    Load Pretrained VGG Model into TensorFlow.
    :param sess: TensorFlow Session
    :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
    :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)
    """
    # TODO: Implement function
    #   Use tf.saved_model.loader.load to load the model and weights
    vgg_tag = 'vgg16'
    vgg_input_tensor_name = 'image_input:0'
    vgg_keep_prob_tensor_name = 'keep_prob:0'
    vgg_layer3_out_tensor_name = 'layer3_out:0'
    vgg_layer4_out_tensor_name = 'layer4_out:0'
    vgg_layer7_out_tensor_name = 'layer7_out:0'
    
    #load model from the path provided
    tf.saved_model.loader.load(sess,[vgg_tag],vgg_path)
    
    #get graph of the loaded model
    savedGraph = tf.get_default_graph()
    
    #get tensors for input , keep_prob ,layre3 output , layer4 output , layer7 output
    image_input= savedGraph.get_tensor_by_name(vgg_input_tensor_name)
    keep_prob=savedGraph.get_tensor_by_name(vgg_keep_prob_tensor_name)
    layer3_out=savedGraph.get_tensor_by_name(vgg_layer3_out_tensor_name)
    layer4_out=savedGraph.get_tensor_by_name(vgg_layer4_out_tensor_name)
    layer7_out=savedGraph.get_tensor_by_name(vgg_layer7_out_tensor_name)
    
    return image_input, keep_prob, layer3_out, layer4_out, layer7_out

tests.test_load_vgg(load_vgg, tf)


Tests Passed


In [4]:
def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    """
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
    :param vgg_layer3_out: TF Tensor for VGG Layer 3 output
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output
    :param vgg_layer7_out: TF Tensor for VGG Layer 7 output
    :param num_classes: Number of classes to classify
    :return: The Tensor for the last layer of output
    """
    
    #To go with transfer learning stop back prorpogation from layer 7
    vgg_layer7_out=tf.stop_gradient(vgg_layer7_out)

    
    #adjust the depth of the layer 7 output to required final classes
    conv_1x1_layer7 = tf.layers.conv2d(vgg_layer7_out,num_classes,1,padding='SAME',kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3),name="modified_conv_1x1_layer7")
    
    #upsample 2x times to the above layer (from x/32 to x/16)
    out_conv_1x1_layer7= tf.layers.conv2d_transpose(conv_1x1_layer7,num_classes,kernel_size=4,strides=(2,2),padding='SAME',kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3),name="modified_out_conv_1x1_layer7")
                                                       
    #for transfer learning stop back propogation from layer 4
    vgg_layer4_out= tf.stop_gradient(vgg_layer4_out)
    
    #use skip connection to add layer 4 output with above
    #but make sure to have same depth by using a 1x1 conv
    
    conv_1x1_layer4 = tf.layers.conv2d(vgg_layer4_out,num_classes,1,padding='SAME',kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3),name="modified_conv_1x1_layer4")
    add_lay4_out_conv_1x1_layer7= tf.add(out_conv_1x1_layer7,conv_1x1_layer4,name="modified_add_lay4_out_conv_1x1_layer7")
    
    #upsample 2x times to the above layer (from x/16 to x/8)
    out_add_layer_7_4= tf.layers.conv2d_transpose(add_lay4_out_conv_1x1_layer7,num_classes,kernel_size=4,strides=(2,2),padding='SAME',kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3),name="modified_out_add_layer_7_4")
    
    #for transfer learning stop back propogation from layer 4
    vgg_layer3_out= tf.stop_gradient(vgg_layer3_out)
    
    #use skip connection to add layer 3 output with above
    #but make sure to have same depth by using a 1x1 conv
    conv_1x1_layer3 = tf.layers.conv2d(vgg_layer3_out,num_classes,1,padding='SAME',kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3),name="modified_conv_1x1_layer3")
    add_lay3_out_add_layer_7_4= tf.add(out_add_layer_7_4,conv_1x1_layer3,name="modified_add_lay3_out_add_layer_7_4")
    
    ##upsample 8x times to the above layer (from x/8 to x) 
    
    lastLayer = tf.layers.conv2d_transpose(add_lay3_out_add_layer_7_4, num_classes,kernel_size=16,strides=(8,8),padding='SAME',kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3),name="modified_lastLayer")
    
    
    
    return lastLayer

tests.test_layers(layers)




Tests Passed


In [5]:
def optimize(nn_last_layer, correct_label, learning_rate, num_classes):
    """
    Build the TensorFLow loss and optimizer operations.
    :param nn_last_layer: TF Tensor of the last layer in the neural network
    :param correct_label: TF Placeholder for the correct label image
    :param learning_rate: TF Placeholder for the learning rate
    :param num_classes: Number of classes to classify
    :return: Tuple of (logits, train_op, cross_entropy_loss)
    """
    # TODO: Implement function
    #reshape from 4d to 2d for ease in classification
    logits = tf.reshape(nn_last_layer, (-1, num_classes),name="modified_logits")
    
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=correct_label)
    
    cross_entropy_loss = tf.reduce_mean(cross_entropy)

    opt = tf.train.AdamOptimizer(learning_rate=learning_rate)
                                 
    train_variables = []

    for variable in tf.trainable_variables():
        # make sure you have named your tensors/ops/layers with names
        # starting with ‘new_’ or some other prefix that you choose
        print("variable.name",variable.name)
        if "modified_" in variable.name or 'beta' in variable.name:
            train_variables.append(variable)
    
    print ("data added")
    train_op = opt.minimize(
                     cross_entropy_loss, name="train_op"
                  )

    
    return logits, train_op, cross_entropy_loss

tests.test_optimize(optimize)




variable.name Variable:0
data added
Tests Passed


In [16]:
def train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate):
    """
    Train neural network and print out the loss during training.
    :param sess: TF Session
    :param epochs: Number of epochs
    :param batch_size: Batch size
    :param get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)
    :param train_op: TF Operation to train the neural network
    :param cross_entropy_loss: TF Tensor for the amount of loss
    :param input_image: TF Placeholder for input images
    :param correct_label: TF Placeholder for label images
    :param keep_prob: TF Placeholder for dropout keep probability
    :param learning_rate: TF Placeholder for learning rate
    """
    # TODO: Implement function
    #sess.run(tf.global_variables_initializer())
    
    my_variable_initializers = [
        var.initializer for var in tf.global_variables() if 'modified_' in var.name or  
               'beta' in var.name  ]
   
    sess.run(my_variable_initializers)

    
    for epoch in range(epochs):
        for X_batch , y_batch in get_batches_fn(batch_size):

            loss, _ = sess.run([cross_entropy_loss, train_op], feed_dict={
                input_image: X_batch,
                correct_label: y_batch,
                keep_prob: keep_prob, # can be between 0 and 1 during training
                learning_rate: learning_rate
            })
        
            print ("EPOCH : " ,epoch, " Loss : " ,loss ) 

 
        

tests.test_train_nn(train_nn)


In [19]:
def run():
    num_classes = 2
    image_shape = (160, 576)
    data_dir = './data'
    runs_dir = './runs'
    tests.test_for_kitti_dataset(data_dir)
    epochs = 3
    batch_size=3

    print("checking for model")
    # Download pretrained vgg model
    helper.maybe_download_pretrained_vgg(data_dir)
    print("model check completed")
    # OPTIONAL: Train and Inference on the cityscapes dataset instead of the Kitti dataset.
    # You'll need a GPU with at least 10 teraFLOPS to train on.
    #  https://www.cityscapes-dataset.com/

    with tf.Session() as sess:
        # Path to vgg model
        vgg_path = os.path.join(data_dir, 'vgg')
        # Create function to get batches
        get_batches_fn = helper.gen_batch_function(os.path.join(data_dir, 'data_road/training'), image_shape)

        # OPTIONAL: Augment Images for better results
        #  https://datascience.stackexchange.com/questions/5224/how-to-prepare-augment-images-for-neural-network

        # TODO: Build NN using load_vgg, layers, and optimize function
        correct_label = tf.placeholder(tf.int32, [None, None, None, num_classes], name='modified_correct_label')
        learning_rate = tf.placeholder(tf.float32, name='modified_learning_rate')
        
        input_layer, keep_prob, layer3, layer4, layer7 = load_vgg(sess, vgg_path)
        
        lastlayerReturned = layers(layer3, layer4, layer7, num_classes)
        logits, train_op, cross_entropy_loss= optimize(lastlayerReturned, correct_label, learning_rate, num_classes)
       
        # TODO: Train NN using the train_nn function
        train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_layer,
             correct_label, keep_prob, learning_rate)    
        
        # TODO: Save inference data using helper.save_inference_samples
        #  helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image)

        # OPTIONAL: Apply the trained model to a video


In [ ]:
if __name__ == '__main__':
    run()